In [1]:
import pandas as pd

In [16]:
# Change this to your local path
df = pd.read_csv(r"C:\Users\bruno\github\SEN1211_project\output\sensitivity.csv", skiprows = 6)

df = df.drop(['alarm?', 'verbose?', 'debug?', 'average-response-time',
         '[step]', 'percentage-stationary-staff', 'max-turtles-per-patch',
         'Preferred-exit-door', 'agents-at-start'],1)

changes = {'precision ((count visitors with [evacuating? = false] / count visitors) * 100) 2': 'percentage_evacuating',
           '[run number]': 'run_number'}

# Renaming relevant column
df = df.rename(columns = changes)
df.head()

,run_number,percentage-female,percentage-children,percentage-visitors-go-to-preferred-exit,alerting-range,evacuation-duration,event-duration,people-in-building,staff-members-in-building,visitors-in-building,percentage_evacuating
0,5,33.3,4.5,86.4,5.4,0,0,450,50,400,100
1,5,33.3,4.5,86.4,5.4,0,1,450,50,400,100
2,5,33.3,4.5,86.4,5.4,0,2,450,50,400,100
3,5,33.3,4.5,86.4,5.4,0,3,450,50,400,100
4,5,33.3,4.5,86.4,5.4,0,4,450,50,400,100


In [28]:
runs = df.groupby(['run_number']).max()
grouped = runs.groupby(['percentage-female',
                        'percentage-children', 
                        'percentage-visitors-go-to-preferred-exit', 
                        'alerting-range'],
                         as_index = False).mean()

grouped = grouped.drop(['people-in-building', 
                        'staff-members-in-building',
                        'event-duration',
                        'visitors-in-building'],1)
grouped.head()

,percentage-female,percentage-children,percentage-visitors-go-to-preferred-exit,alerting-range,evacuation-duration
0,33.3,4.5,86.4,5.4,373.65
1,33.3,4.5,86.4,6.0,357.10
2,33.3,4.5,86.4,6.6,352.40
3,33.3,4.5,96.0,5.4,373.50
4,33.3,4.5,96.0,6.0,366.05


In [30]:
experiments = {'female': [33.3, 37, 40,7],
               'children': [4.5, 5, 5.5],
               'exit': [86.4, 96],
               'alerting-range': [5.4, 6, 6.6]}

In [31]:
base = grouped[(grouped['percentage-female'] == 37) &
        (grouped['percentage-children'] == 5) & 
        (grouped['percentage-visitors-go-to-preferred-exit'] == 96) &
        (grouped['alerting-range'] == 6)]

In [38]:
def get_value(grouped, female, children, exit, alerting):
    base = grouped[(grouped['percentage-female'] == female) &
        (grouped['percentage-children'] == children) & 
        (grouped['percentage-visitors-go-to-preferred-exit'] == exit) &
        (grouped['alerting-range'] == alerting)]
    return float(base['evacuation-duration'].iloc[0])
get_value(grouped, 37, 5, 96, 6)

373.15

In [46]:
sensitivity = {'female -10': get_value(grouped, 33.3, 5, 96, 6),
 'female': get_value(grouped, 37, 5, 96, 6),
 'female +10': get_value(grouped, 40.7, 5, 96, 6),
 'children -10': get_value(grouped, 37, 4.5, 96, 6),
 'children': get_value(grouped, 37, 5, 96, 6),
 'children +10': get_value(grouped, 37, 5.5, 96, 6),
 'exit -10': get_value(grouped, 37, 5, 86.4, 6),
 'exit': get_value(grouped, 37, 5, 96, 6),
 'alerting -10': get_value(grouped, 37, 5, 96, 5.4),
 'alerting': get_value(grouped, 37, 5, 96, 6),
 'alerting +10': get_value(grouped, 37, 5, 96, 6.6),
}
sensitivity

{'female -10': 370.45,
 'female': 373.15,
 'female +10': 362.95,
 'children -10': 372.8,
 'children': 373.15,
 'children +10': 378.9,
 'exit -10': 371.7,
 'exit': 373.15,
 'alerting -10': 368.0,
 'alerting': 373.15,
 'alerting +10': 362.3}

In [57]:
alerting = runs[(runs['percentage-female'] == 37) &
        (runs['percentage-children'] == 5) & 
        (runs['percentage-visitors-go-to-preferred-exit'] == 96)]
alerting[['percentage-female',
          'percentage-children',
          'percentage-visitors-go-to-preferred-exit',
          'alerting-range',
          'evacuation-duration'
         ]]
standard = runs.groupby(['percentage-female',
                        'percentage-children', 
                        'percentage-visitors-go-to-preferred-exit', 
                        'alerting-range'],
                         as_index = False).std()

standard = standard.drop(['people-in-building', 
                        'staff-members-in-building',
                        'event-duration',
                        'visitors-in-building'],1)


standard_dev = {'female -10': get_value(standard, 33.3, 5, 96, 6),
 'female': get_value(standard, 37, 5, 96, 6),
 'female +10': get_value(standard, 40.7, 5, 96, 6),
 'children -10': get_value(standard, 37, 4.5, 96, 6),
 'children': get_value(standard, 37, 5, 96, 6),
 'children +10': get_value(standard, 37, 5.5, 96, 6),
 'exit -10': get_value(standard, 37, 5, 86.4, 6),
 'exit': get_value(standard, 37, 5, 96, 6),
 'alerting -10': get_value(standard, 37, 5, 96, 5.4),
 'alerting': get_value(standard, 37, 5, 96, 6),
 'alerting +10': get_value(standard, 37, 5, 96, 6.6),
}
standard_dev

{'female -10': 23.027386441274434,
 'female': 25.047901477390326,
 'female +10': 25.413371946363398,
 'children -10': 25.8143167131811,
 'children': 25.047901477390326,
 'children +10': 15.081881775651762,
 'exit -10': 20.039697444926926,
 'exit': 25.047901477390326,
 'alerting -10': 29.947322171826272,
 'alerting': 25.047901477390326,
 'alerting +10': 28.15576971814219}